In [ ]:
!git clone https://github.com/Francesco9932/rag_risiko

In [ ]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q -U bitsandbytes
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

!pip install accelerate
!pip install -q pypdf
!pip install -q spacy
!python3 -m spacy download it_core_news_sm

In [3]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

import os

In [39]:
from langchain.document_loaders import PyPDFLoader

pdf_folder_path = "./rag_risiko/regolamenti/"
documents = []
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())

In [40]:
print(documents[0].page_content)

EDIZIONE PER 2 GIOCATORI!© EXPLODING KITTENS 2023EXPLODING KITTENS
REGOLAMENTO
2 GIOCATORI
CONTIENE: 32 CARTE
CRUNCH
MUNCH
MUNCH
CRUNCH
MUNCH
MUNCH
SVOLGIMENTO DEL GIOCO
Il mazzo di carte contiene 1 carta Exploding 
Kitten. La partita inizia mettendo il mazzo 
al centro del tavolo a faccia in giù. Ogni 
giocatore, a turno, pesca una carta finché un 
giocatore non pesca la carta Exploding Kitten.
Quando questo accade, quel giocatore 
esplode ed è eliminato dal gioco.
Il giocatore rimanente vince la partita.
Tutte le altre carte forniscono potenti 
strumenti per evitare di esplodere!
IN BREVE
SE ESPLODETE, AVETE PERSO
E SARETE PERVASI DA BILE INCENDIARIA DA PERDENTE.
SE NON ESPLODETE, AVETE VINTO
E SARETE PIENI DI GLORIA. OTTIMO LAVORO.
TUTTE LE ALTRE CARTE
RIDURRANNO LE VOSTRE PROBABILITÀ DI SALTARE  
IN ARIA A CAUSA DI UNA CARTA EXPLODING KITTEN.
PER ESEMPIO
Nel vostro turno potete giocare una 
carta Prevedi il Futuro  per guardare 
le carte in cima al mazzo prima di 
pescare.
Se così 

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter()
docs = splitter.split_documents(documents)

In [42]:
docs[2]

Document(page_content="NON PESCARE ALCUNA CARTA.  \nOBBLIGA IL GIOCATORE SUCCESSIVO  \nA SVOLGERE DUE TURNI.SCATENA IL MICIARAMPA.attaccoattacco(   )2x(   )2x\n2x\n2x\nAndando a fare i bisognini nel wormhole  \ndelle mille possibilita mandero in \nfrantumi l’universo conosciuto.’’\nInoltre, puzzera \nun pochino.’MESCOLAMESCOLAMESCOLA IL MAZZO DI PESCA.\nEDIZIONE PER 2 GIOCATORICOMPARE UNA LETTIERA  \nTRANSDIMENSIONALE.\nPREVEDI IL FUTURO  (3X)PREVEDI IL FUTURO  (3X)\nEDIZIONE PER 2 GIOCATORI\n3x\n3xSGUINZAGLIA I TUOI CONIGLI-SPIA.\nGUARDA SEGRETAMENTE LE PRIME  TRE CARTE DEL MAZZO DI PESCA.\nEDIZIONEORIGINALENO!NO!UN NO-NJA SFERRA  \nUN MALEFICO CALCIO VOLANTE.\nBLOCCA L’AZIONE DI UN ALTRO GIOCATORE.  \nPUOI GIOCARE QUESTA CARTA IN QUALSIASI MOMENTO.ESEMPIO DI TURNO\nSOSPETTI CHE LA CARTA IN CIMA AL MAZZO DI PESCA SIA UNA \nCARTA “EXPLODING KITTEN”, QUINDI INVECE DI PASSARE E POI \nPESCARE 1 CARTA PER TERMINARE IL TUO TURNO, DECIDI DI GIOCARE \n1 CARTA “PREVEDI IL FUTURO”, CHE TI PERME

In [43]:
# Define the path to the pre-trained model you want to use
modelPath = "nickprock/sentence-bert-base-italian-uncased"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
# use cpu or gpu
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_kwargs = {'device': device}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [44]:
db = FAISS.from_documents(docs, embeddings)

In [11]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [12]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [13]:
# template="""
#
#
#                             \n\n\nPlayer Question:\n{input}"""

template_ita = """
[INST] <>

Sei un esperto giocatore e insegnante di giochi da tavolo. Hai memorizzato tutte le regole di ogni gioco da tavolo mai creato.
Utilizza le regole nella tua memoria insieme a qualsiasi altro contesto che possiedi per rispondere alle domande dei giocatori qui sotto.
<>

{context}
#
{question} [/INST]
"""


template = """
[INST] <>
You are an expert board game player and teacher. You have memorized all the rules to every board game ever made.
Use the rules in your memory along with any other context you have to answer the players questions below.
<>

{context}
#
{question} [/INST]
"""

In [45]:
from langchain import PromptTemplate

prompt = PromptTemplate(template=template, input_variables=["context", "input"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [17]:
input = "Quanti sono i territori di risiko? Dammi una risposta coincisa con solo il numero di territori."
result_ = qa_chain(
    input
)
result = result_["result"].strip()

print(input)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Quanti sono i territori di risiko? Dammi una risposta coincisa con solo il numero di territori.
There are 42 territories of risk in Risk!


In [22]:
input = "Riassumi lo scopo del gioco risiko, specificandone i punti chiave e numero massimo di giocatori. Devi essere coinciso"
result_ = qa_chain(
    input
)
result = result_["result"].strip()


print(input)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Riassumi lo scopo del gioco risiko, specificandone i punti chiave e numero massimo di giocatori. Devi essere coinciso
Risiko is a strategic board game where players compete for world domination by occupying territories and eliminating opponents. The goal of the game is to achieve one's secret objective before the others. Each player chooses a color for their armies and rolls a die to determine who goes first. Players take turns rolling dice and moving their armies across the map, attacking or defending territories as needed. The game ends when a player achieves their secret objective or all other players have been eliminated. Risiko is designed for 3-6 players.


In [24]:
input = "Mi spieghi come funziona il combattimento nel gioco risiko in inglese? approfondisci il caso di pareggio"
result_ = qa_chain(
    input
)
result = result_["result"].strip()


print(input)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Mi spieghi come funziona il combattimento nel gioco risiko in inglese? approfondisci il caso di pareggio
In Risiko, combat is resolved by rolling dice. Each die represents one of three different types of weapons: attack, defense, or neutral. The number on each die determines how many hits or misses occur during the attack. If both sides roll the same number, it's considered a tie and the attacker gets to roll again. If the attacker rolls higher than the defender, they win the battle and gain control of the territory. If the defender rolls higher, they win and defend their territory. If both sides roll the same number, it's considered a stalemate and no winner is declared.


In [34]:
# Without External Knowledge: Exploding Kittens

input = "Come funziona il gioco Exploding Kittens?"
result_ = qa_chain(
    input
)
result = result_["result"].strip()


print(input)
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Come funziona il gioco Exploding Kittens? mi dici da dove hai preso questa informazione
Exploding Kittens is a card game that was created by Max Funke and Scott Fischer. The game was first released on Kickstarter in 2015 and quickly gained popularity among gamers. I got my information about the game from various online sources such as reviews, tutorials, and official product pages.


In [46]:
# With External Knowledge: Exploding Kittens

input = "Come funziona il gioco Exploding Kittens?"
result_ = qa_chain(
    input
)
result = result_["result"].strip()


print(input)
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Come funziona il gioco Exploding Kittens? mi dici da dove hai preso questa informazione
The game of Exploding Kittens is a card game that can be played by two or more players. The goal of the game is to be the last player remaining in the game without having an "Exploding Kitten" card in their hand. Each player starts with a deck of cards, including one "Exploding Kitten" card. During each turn, players draw one card from the deck and play it if they choose to. Some cards allow players to perform actions such as skipping turns, swapping hands, or drawing additional cards. Other cards may force opponents to take specific actions or lose points. If a player draws an "Exploding Kitten" card during their turn, they immediately lose the game.


In [ ]:
# # Questa parte da rivedere:

# # Specify the model name you want to use
# model_name = "mrm8488/bert-italian-finedtuned-squadv1-it-alfa"

# # Load the tokenizer associated with the specified model
# tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# # Define a question-answering pipeline using the model and tokenizer
# question_answerer = pipeline(
#     "question-answering",
#     model=model_name,
#     tokenizer=tokenizer,
#     return_tensors='pt'
# )

# # Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# # with additional model-specific arguments (temperature and max_length)
# llm = HuggingFacePipeline(
#     pipeline=question_answerer
# )

Some weights of the model checkpoint at mrm8488/bert-italian-finedtuned-squadv1-it-alfa were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# # Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
# retriever = db.as_retriever()

# # Create a question-answering instance (qa) using the RetrievalQA class.
# # It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
# qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [ ]:
# question = "quanti sono i territori?"
# result = qa.run({"query": question})
# print(result["result"])